In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

# 假设已经加载生成的数据
data = pd.read_csv('data_200_samples.csv')

In [2]:
# 1）将特征X送入随机森林训练估计每个样本的倾向评分。然后根据倾向评分进行样本匹配，控制治疗组和对照组协变量一致。
X = data[['X1', 'X2', 'X3']]
W = data['W']

# 使用随机森林分类器估计倾向评分
ps_model = RandomForestClassifier(n_estimators=100, random_state=42)
ps_model.fit(X, W)
data['e'] = ps_model.predict_proba(X)[:, 1]


In [3]:
# 2）通过将特征x送入随机森林，估计没有治疗情况下的潜在结果。
# 使用随机森林回归器估计潜在结果
y0_model = RandomForestRegressor(n_estimators=100, random_state=42)
y1_model = RandomForestRegressor(n_estimators=100, random_state=42)

# 根据未接受治疗（W=0）的样本训练潜在结果模型
y0_model.fit(X[W == 0], data[W == 0]['Y'])
# 根据接受治疗（W=1）的样本训练潜在结果模型
y1_model.fit(X[W == 1], data[W == 1]['Y'])

# 估计每个样本的潜在结果
data['m0'] = y0_model.predict(X)
data['m1'] = y1_model.predict(X)

In [4]:
# 3）然后将估计得到的样本倾向评分和潜在结果代入R-learner公式（保证双重稳健因果效应），并化为回归最小化问题，从而估计出CATE治疗效果。最后进行讨论验证，如对不同子群体验证异质性等。
# 计算双重稳健因果效应估计值
data['DR'] = ((data['W'] / data['e'] - (1 - data['W']) / (1 - data['e'])) * 
              (data['Y'] - data['W'] * data['m1'] - (1 - data['W']) * data['m0']) + 
              data['m1'] - data['m0'])

# 计算平均治疗效果（CATE）
CATE_estimate = data['DR'].mean()

# 打印估计的CATE
print(f'Estimated CATE: {CATE_estimate}')

Estimated CATE: 1.1066073910483005


In [5]:
# 进一步的验证，如子群分析，验证异质性
# 例如，针对特定子群（如X1大于中位数的样本）计算CATE
subgroup = data[data['X1'] > data['X1'].median()]
subgroup_CATE_estimate = subgroup['DR'].mean()
print(f'Subgroup CATE estimate (X1 > median): {subgroup_CATE_estimate}')


Subgroup CATE estimate (X1 > median): 1.1345554679026002
